### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Hemochromatosis/GSE129322'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# 1. Gene Expression Data Availability
# From the Series summary, RT-PCR indicates that gene expression data is available.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Referring to the Sample Characteristics Dictionary:
# - Hemochromatosis data is available under key 8.
# - Age data is available under key 1.
# - Gender data is available under key 0.

trait_row = 8
age_row = 1
gender_row = 0

# 2.3 Data Type Conversion
def convert_trait(value):
    try:
        val = value.split(': ')[1]
        if val == 'yes':
            return 1
        elif val == 'no':
            return 0
        else:
            return None
    except IndexError:
        return None
      
def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except (IndexError, ValueError):
        return None

def convert_gender(value):
    try:
        val = value.split(': ')[1]
        if val == 'F':
            return 0
        elif val == 'M':
            return 1
        else:
            return None
    except IndexError:
        return None

# 3. Save Metadata
save_cohort_info('GSE129322', './preprocessed/Hemochromatosis/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Hemochromatosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Hemochromatosis/trait_data/GSE129322.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])
    

# Given the empty index from the output of STEP3, we cannot infer the type of identifiers.
# To be safe and ensure proper analysis, we assume gene mapping is required.


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Determine identifier key and gene symbol key from the gene annotation dictionary
identifier_key = 'Probe Set ID'
gene_symbol_key = 'ORF'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# Inspecting the structure of gene_data before any modifications
print("Initial gene_data columns:", gene_data.columns)
print("Initial gene_data head:", gene_data.head())

# 3. Ensure `gene_data` has the correct column setup before applying the mapping
gene_data.reset_index(inplace=True)  # Ensure it is a column
gene_data.columns = ['ID'] + list(gene_data.columns[1:])  # Rename for consistency

# Verify the structure after resetting the index and renaming columns
print("Modified gene_data columns:", gene_data.columns)
print("Modified gene_data head:", gene_data.head())

# Apply the gene mapping function correctly
gene_data = apply_gene_mapping(gene_data.set_index('ID'), gene_mapping)

# Print gene_data to verify it applied correctly
print("Mapped gene_data head:", gene_data.head())


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Hemochromatosis/gene_data/GSE129322.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Hemochromatosis')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE129322', './preprocessed/Hemochromatosis/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Hemochromatosis/GSE129322.csv'
    unbiased_merged_data.to_csv(csv_path)
